# Example State Machine Usage

## Hierarchical State Machine

Demonstrating use of the CompositeState class.

In [31]:
from state_machine import StateMachine, State, Event, CompositeState

In [32]:
from state_machine import Event, State, StateMachine, CompositeState, FinalState
from mock import MagicMock


chatbot = StateMachine('chatbot')
active_state = chatbot.create_state('active', CompositeState)
sleeping_state = chatbot.create_state('sleeping', State)

chatbot.initial_state=active_state
active_state.add_transition_to(sleeping_state, 'sunset')
sleeping_state.add_transition_to(active_state, 'sunrise')

INFO:default:Creating state of type <class 'state_machine.composite_state.CompositeState'>
INFO:default:Creating state of type <class 'state_machine.state.State'>


In [33]:
def become_happy( event, state ):
    state.logger.info('Becoming Happy')
    
def happy_message(event, state):
    if event.name=='msg':
        state.logger.info('Hello %s, I am happy', event.payload['from'])
    
def become_sad(event, state):
    state.logger.info('Becoming Sad')
    
def sad_message(event, state):
    if event.name=='msg':
        state.logger.info('Hello %s, I am sad', event.payload['from'])

happy_state = active_state.create_state('happy')
happy_state.on_start = become_happy
happy_state.on_run   = happy_message

sad_state = active_state.create_state('sad')
sad_state.on_start = become_sad
sad_state.on_run = sad_message

happy_state.add_transition_to(sad_state, 'criticism')
sad_state.add_transition_to(happy_state, 'praise')

active_state.initial_state=happy_state
active_state.reset()

sunrise=Event('sunrise')
sunset=Event('sunset')

hello = Event('msg',{'from':'Bernie'})

criticism=Event('criticism')
praise=Event('praise')

INFO:default:Creating state of type <class 'state_machine.state.State'>
INFO:default:Creating state of type <class 'state_machine.state.State'>


In [34]:
chatbot.reset()

On sunset, the chat bot goes to sleep, so criticism and praise have no affect on it.

In [35]:
# Send the bot to sleep
chatbot.notify(sunset)

#It is unresponsive
chatbot.notify(criticism)
chatbot.notify(hello)
chatbot.notify(praise)
chatbot.notify(hello)

INFO:default:chatbot transitioning to state State:sleeping


On sunrise, the chatbot wakes. so criticism or praise change the internal state.

In [36]:
# Wake up the chat bot
chatbot.notify(sunrise)

# It is responsive
chatbot.notify(criticism)
chatbot.notify(hello)
chatbot.notify(praise)
chatbot.notify(hello)

INFO:default:chatbot transitioning to state State:active
INFO:default:active transitioning to state State:sad
INFO:default:Becoming Sad
INFO:default:Hello Bernie, I am sad
INFO:default:active transitioning to state State:happy
INFO:default:Becoming Happy
INFO:default:Hello Bernie, I am happy
